In [109]:
import math
import pandas as pd

df = pd.read_csv('data/closing_price.csv')

selisih_mutlak = []
close_list = df['Close'].tolist()

for i in range(len(close_list)):
    next_pointer = i + 1
    selisih_mutlak.append(abs(close_list[next_pointer] - close_list[i]))

    if next_pointer >= len(close_list) - 1:
        break

jumlah_data = df['Close'].count()
jumlah_interval_k = 1 + (3.3 * math.log10(jumlah_data))
panjang_interval = (df['Close'].max() - df['Close'].min()) / jumlah_interval_k

print(f"Jumlah Data: {jumlah_data}")
print(f"Jumlah Interval: {jumlah_interval_k}")
print(f"Panjang Interval: {panjang_interval}")

batas_bawah = [df['Close'].min()]
batas_atas = []
batas_bawah_atas = []

for i in range(round(jumlah_interval_k)):
    batas_atas.append(batas_bawah[i] + panjang_interval)
    batas_bawah.append(batas_atas[i])
    batas_bawah_atas.append([batas_bawah[i], batas_atas[i]])

print(f"Batas bawah: {batas_bawah}")
print(f"Batas atas: {batas_atas}")
print(f"Batas bawah dan atas: {batas_bawah_atas}")

nilai_tengah = []
for v in batas_bawah_atas:
    nilai_tengah.append((v[0] + v[1]) / 2)

print(f"Nilai tengah: {nilai_tengah}")

derajat_anggota = []
for i in range(round(jumlah_interval_k)):
    temp = []
    for j in range(round(jumlah_interval_k)):
        if j == i:
            temp.append(1)
        elif j == i - 1 or j == i + 1:
            temp.append(0.5)
        else:
            temp.append(0)
    derajat_anggota.append(temp)

print(f"Derajat Anggota: {derajat_anggota}")

fuzifikasi_ai = []
for index, price in enumerate(close_list):
    for i, interval in enumerate(batas_bawah_atas):
        if interval[0] <= price <= interval[1]:
            fuzifikasi_ai.append(i + 1)
            break

print(f"fuzifikasi ai: {fuzifikasi_ai}")
print(f"Len fuzifikasi ai: {len(fuzifikasi_ai)}")

FLR_group = {}
for i in range(len(fuzifikasi_ai)):
    next_pointer = i + 1

    current_stage = fuzifikasi_ai[i]
    next_stage = fuzifikasi_ai[next_pointer]

    if current_stage in FLR_group:
        FLR_group[current_stage].append(next_stage)
    else:
        FLR_group[current_stage] = [next_stage]

    if next_pointer >= len(fuzifikasi_ai) - 1:
        break

print(f"FLR Group: {FLR_group}")

group = {}
for key, v in FLR_group.items():
    group[key] = {}
    for next_stage in v:
        if next_stage in group[key]:
            group[key][next_stage] += 1
        else:
            group[key][next_stage] = 1

print(f"Group: {group}")

nilai_defuzzifikasi = {}
for key, value in group.items():
    a = 0
    divider = 0
    for k, v in value.items():
        a += nilai_tengah[k - 1] * v
        divider += v

    nilai_defuzzifikasi[key] = a / divider

print(f"Nilai Defuzzifikasi: {nilai_defuzzifikasi}")

hasil_prediksi = []
for i, v in enumerate(fuzifikasi_ai):
    if i > 0:
        hasil_prediksi.append(nilai_defuzzifikasi[v])
    
print(f"Hasil Prediksi: {hasil_prediksi}")
print(f"Hasil Prediksi: {len(hasil_prediksi)}")

selisih_data_aktual = []
for i, price in enumerate(close_list):
    next_pointer = i + 1
    
    a = ((close_list[next_pointer] - hasil_prediksi[i]) / close_list[next_pointer])
    selisih_data_aktual.append(abs(a))
    
    if next_pointer >= len(close_list) - 1:
        break

print(f"Selisih Data: {selisih_data_aktual}")

mape = sum(selisih_data_aktual) / len(selisih_data_aktual)
print(f"Mape: {round(mape, 5)}")


Jumlah Data: 259
Jumlah Interval: 8.963889221468131
Panjang Interval: 51.392863032788426
Batas bawah: [624.76001, 676.1528730327884, 727.5457360655769, 778.9385990983653, 830.3314621311538, 881.7243251639422, 933.1171881967307, 984.5100512295191, 1035.9029142623076, 1087.295777295096]
Batas atas: [676.1528730327884, 727.5457360655769, 778.9385990983653, 830.3314621311538, 881.7243251639422, 933.1171881967307, 984.5100512295191, 1035.9029142623076, 1087.295777295096]
Batas bawah dan atas: [[624.76001, 676.1528730327884], [676.1528730327884, 727.5457360655769], [727.5457360655769, 778.9385990983653], [778.9385990983653, 830.3314621311538], [830.3314621311538, 881.7243251639422], [881.7243251639422, 933.1171881967307], [933.1171881967307, 984.5100512295191], [984.5100512295191, 1035.9029142623076], [1035.9029142623076, 1087.295777295096]]
Nilai tengah: [650.4564415163942, 701.8493045491826, 753.2421675819711, 804.6350306147596, 856.027893647548, 907.4207566803365, 958.8136197131249, 1010.